In [1]:
import streamlit as st
import speech_recognition as sr
import requests
import random
import textstat
from openai import OpenAI  # Importing OpenAI to connect with LM Studio

In [4]:
# Define feedback function using LM Studio
def analyze_response(response_text, behavior_role, role, response_criteria):
    # Analyze brevity, filler words, and repetition using textstat
    feedback = {
        "brevity": f"Reading ease score: {textstat.flesch_reading_ease(response_text)}",
        "filler_words": "No obvious filler words detected"  # Placeholder for a better detection mechanism
    }

    # Request feedback from LM Studio
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
    prompt = f"As a {behavior_role}, imagine you are taking an interview for the role {role}, analyze this answer: '{response_text}'. Focus on: {response_criteria}"
    
    response = client.chat.completions.create(
        model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
        messages=[
            {"role": "system", "content": "You are an intelligent assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        stream=True,
    )
    
    new_message = {"role": "assistant", "content": ""}
    
    for chunk in response:
        if chunk.choices[0].delta.content:
            new_message["content"] += chunk.choices[0].delta.content

    feedback['LLM_feedback'] = new_message['content'].strip()
    
    return feedback

In [5]:
response_text = """• Live in the future: Let’s build the future of work that we want, with an in-person focus designed to support a strong, valuable experience for our people who have chosen to work from the office, and a thoughtful and intentional approach to where we invest in remote work. This also means being early adopters of the future products we build to help people feel present together wherever they are.
"""
behavior_role = st.text_input("Define the role of the LLM (e.g., HR recruiter):", "HR recruiter")
response_criteria = st.text_area(
    "Describe the type of response needed (e.g., brevity, time taken, quality):", 
    "Provide feedback on time taken, brevity, filler words, repetition, and answer quality."
)
role = 'ML Manager'
analyze_response(response_text, behavior_role, role, response_criteria)

2024-10-27 22:07:17.994 
  command:

    streamlit run /Users/rshankar/anaconda3/envs/cv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


{'brevity': 'Reading ease score: 53.38',
 'filler_words': 'No obvious filler words detected',
 'LLM_feedback': 'Here\'s my analysis:\n\n**Time taken:** The candidate takes approximately 20-25 seconds to provide their response. This is a good pace, but it could be slightly faster considering the question seems to allow for a more open-ended response.\n\n**Brevity:**\nThe candidate\'s answer is somewhat lengthy (around 45 words) and doesn\'t immediately get to the point. They start with an introductory phrase ("Let\'s build...") that sets up their idea, but then meanders slightly before getting to the core of their response. A more direct and concise approach would be beneficial.\n\n**Filler words:**\nThe candidate uses a few filler words/phrases ("this also means," "with an in-person focus designed to support"), which can make the answer feel less polished.\n\n**Repetition:**\nThere is no clear repetition in this answer, so I\'ll mark it as "0" on that metric.\n\n**Answer quality:**\n\n